resize selenium window?

In [ ]:

var rows = 6;
var columns = 6;
var screen = {
    height: 1024 * 6,
    width: 1350 * 6
};

function getScreenSize() {
    return client
        .windowHandle()
        .then(r => client.windowHandleMaximize(r.value))
        .catch(e => {})
        .windowHandleSize()
        .then(r => (screen = r.value))
        .catch(e => console.log(e))
}

function resizeWindow(i = 0) {
    i = Math.abs(i + (typeof OFFSET !== 'undefined' ? OFFSET : 0));
    var hwnd;
    return client
        .windowHandle()
        .then(r => client.window((hwnd = r.value)))
        .then(() => typeof screen === 'undefined' ? getScreenSize() : screen)
        .catch(e => console.log(e))
        .then(screen => {
            const newPosition = {
                x: Math.floor(i % columns)
                    * Math.floor(screen.width / columns),
                y: Math.floor(i / columns % rows)
                    * Math.floor(screen.height / rows)
            };
            console.log(newPosition);
            return client.windowHandlePosition(hwnd, newPosition)
        })
        .catch(e => console.log(e))
        .then(() => client.windowHandleSize(hwnd, {
            width: Math.floor(screen.width / columns),
            height: Math.floor(screen.height / rows)
        }))
        .catch(e => console.log(e))
}

if(typeof client.resizeWindow === 'undefined') {
    client.addCommand('resizeWindow', resizeWindow);
}
module.exports = resizeWindow;


only one window?



In [ ]:
var importer = require('../Core');

function closeAllTabs(keep) {
    return client
        .getTabIds()
        .then(h => {
            var promises = [];
            for(var i = 0; i < h.length; i++) {
                if(h[i] === keep) {
                    continue;
                }
                promises.push((i => resolve => client
                              .switchTab(h[i])
                              .close()
                              .then(() => resolve())
                              .catch(e => resolve()))
                              .apply(this, [i]));
            }
            return importer.runAllPromises(promises);
        })
        .catch(e => console.log(e))
}

function closeAllWindows(keep) {
    return client
        .windowHandles()
        .then(h => {
            var promises = [];
            for(var i = 0; i < h.value.length; i++) {
                if(h.value[i] === keep) {
                    continue;
                }
                promises.push((i => resolve => client
                              .window(h.value[i])
                              .close()
                              .then(() => resolve())
                              .catch(e => resolve()))
                              .apply(this, [i]));
            }
            return importer.runAllPromises(promises);
        })
        .window(keep)
        .catch(e => console.log(e))
}

function onlyOneWindow() {
    return client
        .newWindow('https://google.com')
        .windowHandle()
        .then(r => closeAllWindows(r.value))
        .getCurrentTabId()
        .then(r => closeAllTabs(r))
        .catch(e => console.log(e))
    // TODO: close all tabs
}

module.exports = onlyOneWindow;



tile selenium chrome windows?



In [ ]:
var importer = require('../Core');
var {getAllSessionUrls} = importer.import(['resize selenium window', 'get all session urls'], {client})

function openUrl(url) {
    var handles;
    return client
        .windowHandles()
        .then(h => handles = h.value)
        .execute((url, width, height) => {
            window.open(url, '', 'width=1350,height=1024');
        }, url)
        .windowHandles()
        .then(h => h.value.filter(hwnd => handles.indexOf(hwnd) === -1)[0])
        .catch(e => console.log(e))
}

function createNewWindows(urls) {
    var hwnd;
    return client
        .windowHandle()
        .then(r => hwnd = r.value)
        .windowHandles()
        .then(r => {
            // -1 first window is already used by sliced url
            const count = urls.length - (r.value.length - 1);
            const promises = [];
            for(var i = 0; i < count; i++) {
                promises.push((i => resolve => {
                    console.log(urls[i]);
                    return openUrl(urls[i])
                        .window(hwnd)
                        .then(h => resolve(h))
                        .catch(e => { console.log(e); resolve() })
                }).apply(this, [i]));
            }
            return importer.runAllPromises(promises);
        })
        .catch(e => console.log(e))
}

function tileWindows(urls) {
    return client
        .resizeWindow(0)
        .url(typeof urls === 'string' ? urls : urls[0])
        .then(h => typeof urls === 'string' ? [] : createNewWindows(urls.slice(1)))
        .then(() => getAllSessionUrls())
        .windowHandles()
        .then(r => r.value)
        .catch(e => console.log(e))
}
module.exports = tileWindows;



In [ ]:
var importer = require('../Core');
var runSeleniumCell = importer.import('selenium cell');

var query = 'list of search engines';

$$.async();
// get multiple sets of 9 working for single page scraping in parallel
runSeleniumCell('tile chrome windows')
    .then(tileWindows => tileWindows([
        'https://www.google.com/search?q=' + query,
        'https://www.bing.com/search?q=' + query,
        'https://search.yahoo.com/search?p=' + query,
        'https://www.ask.com/web?q=' + query,
        'https://search.aol.com/aol/search?q=' + query,
        'https://www.baidu.com/s?wd=' + query,
        'https://www.wolframalpha.com/input/?i=' + query,
        'https://duckduckgo.com/?q=' + query,
        'https://archive.org/search.php?query=' + query,
    ]))
    .then(r => $$.sendResult(r))
    .catch(e => $$.sendError(e))



get all session and window urls?



In [ ]:
var importer = require('../Core');

function getAllSessionUrls(reposition = true) {
    var windowCounter = -1;
    var session = client.requestHandler.sessionID;
    return client
        .getActiveSessions()
        .then(r => {
            console.log(r);
            return importer.runAllPromises(r.map(s => resolve => {
                client.requestHandler.sessionID = s;
                return client
                    .getUrl()
                    .windowHandles()
                    .then(h => importer.runAllPromises(h.value.map(hwnd => resolve => {
                        windowCounter++
                        return client
                            .window(hwnd)
                            .then(() => reposition ? resizeWindow(windowCounter) : [])
                            .getUrl()
                            .then(r => resolve(r))
                            .catch(e => console.log(e))
                    })))
                    .then(r => resolve(r))
                    .catch(e => console.log(e))
            }))
        })
        .then(r => {
            client.requestHandler.sessionID = session;
            return r;
        })
        .catch(e => console.log(e))
}

module.exports = getAllSessionUrls;


In [ ]:
var importer = require('../Core');
var runSeleniumCell = importer.import('selenium cell');

$$.async();
runSeleniumCell(['resize selenium window', 'get all session urls'], false)
    .then(r => r.getAllSessionUrls(false))
    .then(r => $$.sendResult(r))
    .catch(e => $$.sendError(e))


send a joke?

In [ ]:
var util = require('util');
var request = util.promisify(require('request'));

function regexToArray(ex, str, i = 0) {
    var co = []; var m;
    while ((m = ex.exec(str)) && co.push(m[i]));
    return co;
}

var jokes;
function getJoke() {
    return (typeof jokes === 'undefined'
        ? request('http://www.ducksters.com/jokes/silly.php')
        .then(res => regexToArray(/^.*?Q:.*$|^.*?A:.*$/igm, res.body))
        .then(r => {
            r = r.reduce((arr, j, i) => {
                if(i % 2 === 1) {
                    arr.push([
                        r[i-1].replace(/<.*?\s*\/?>/ig, '').trim().replace(/^\s*|\s*$/igm, ''),
                        j.replace(/<.*?\s*\/?>/ig, '').trim().replace(/^\s*|\s*$/igm, '')
                    ]);
                }
                return arr;
            }, []);
            console.log(r);
            jokes = r;
            return r;
        })
        : Promise.resolve(jokes))
        .then(arr => {
            const i = Math.round(Math.random() * arr.length);
            return arr[i];
        })
}

function sendJoke(hwnd) {
    var joke;
    return client
        .then(() => getJoke())
        .then(r => joke = r)
        .window(hwnd)
        .then(() => sendFacebookMessage(joke[0]))
        .pause(20000)
        .then(() => sendFacebookMessage(joke[1]))
}
module.exports = sendJoke;


send facebook thanks?



In [ ]:
function sendFacebookThanks(friend, hwnd) {
    return client
        .window(hwnd)
        .clickSpa(friend)
        .then(() => readFacebookThread(friend))
        .then(r => {
            const thanks = r.messages.filter(m => m.message.indexOf('dream') > -1);
            console.log(thanks);
            return thanks.length === 0
              ? sendFacebookMessage('Are you living the dream?')
              : []
        })
        .catch(e => console.log(e))
}
module.exports = sendFacebookThanks;


Monitor many chats at once and continuously


In [1]:
var importer = require('../Core');
var runSeleniumCell = importer.import('selenium cell');

var friends = [
    'https://www.facebook.com/messages/t/patti.caudill',
    'https://www.facebook.com/messages/t/andrei.darie.5',
    'https://www.facebook.com/messages/t/setha.vanderhoof',
    /*
    'https://www.facebook.com/messages/t/Lloydster4',
    'https://www.facebook.com/messages/t/jessegill',
    'https://www.facebook.com/messages/t/lisa.potter.7140',
    'https://www.facebook.com/messages/t/StevenLockhart123',
    'https://www.facebook.com/messages/t/lidia.ws.7',
    'https://www.facebook.com/messages/t/corinne.cullinan',
    'https://www.facebook.com/messages/t/andreas.stylianou.98',
    'https://www.facebook.com/messages/t/robert.holweger',
    'https://www.facebook.com/messages/t/tintin.katy512',
    'https://www.facebook.com/messages/t/greg.maserati.7',
    'https://www.facebook.com/messages/t/scott.johnson.31149359',
    'https://www.facebook.com/messages/t/railhunk',
    'https://www.facebook.com/messages/t/jeff.coffman.92',
    'https://www.facebook.com/messages/t/robert.holweger',
    'https://www.facebook.com/messages/t/100015169534875',
    'https://www.facebook.com/messages/t/chase.mcvey.9',
    'https://www.facebook.com/messages/t/ryan.raub',
    'https://www.facebook.com/messages/t/brian.harris.01',
    'https://www.facebook.com/messages/t/borikoni',
    'https://www.facebook.com/messages/t/tim.bangert.3',
    'https://www.facebook.com/messages/t/jsims730',
    'https://www.facebook.com/messages/t/Ryan-A-Stoffko-1784292651894577',
    'https://www.facebook.com/messages/t/sonu.karve.94',
    'https://www.facebook.com/messages/t/jason.hornack',
    'https://www.facebook.com/messages/t/BumpyJonson',
    'https://www.facebook.com/messages/t/frank.vitale01',
    'https://www.facebook.com/messages/t/jeff.kalina.79',
    'https://www.facebook.com/messages/t/demi.s.jiang',
    'https://www.facebook.com/messages/t/stephen.baier',
    'https://www.facebook.com/messages/t/mietusmm',
    'https://www.facebook.com/messages/t/chris.huie.14',
    'https://www.facebook.com/messages/t/vitali.kaush',
    'https://www.facebook.com/messages/t/FJoeAZ',
    'https://www.facebook.com/messages/t/SusanAgresta',
    'https://www.facebook.com/messages/t/andrew.s.mcvey.3',
    'https://www.facebook.com/messages/t/faulkner.shane',
    'https://www.facebook.com/messages/t/100013570236133',
    'https://www.facebook.com/messages/t/ryan.stoffko',
    'https://www.facebook.com/messages/t/6212130467169267713',
    'https://www.facebook.com/messages/t/joanna.street24',
    'https://www.facebook.com/messages/t/stephanie.desertgirl',
    'https://www.facebook.com/messages/t/vladimir.zuzukin',
    'https://www.facebook.com/messages/t/ScientificMaster.Me',
    'https://www.facebook.com/messages/t/maungkyarphyuu',
    'https://www.facebook.com/messages/t/maciejc',
    'https://www.facebook.com/messages/t/travis.hudson.507',
    'https://www.facebook.com/messages/t/ryan.raub',
    'https://www.facebook.com/messages/t/jennifer.reaves.58',
    'https://www.facebook.com/messages/t/jeff.kalina.79',
    'https://www.facebook.com/messages/t/talbert.tso',
    'https://www.facebook.com/messages/t/greg.maserati.7',
    'https://www.facebook.com/messages/t/katya.geissler',
    'https://www.facebook.com/messages/t/100008264283932',
    'https://www.facebook.com/messages/t/frederick.murphy.56',
    'https://www.facebook.com/messages/t/april.knapp.13',
    'https://www.facebook.com/messages/t/carisa.knight6000',
    'https://www.facebook.com/messages/t/Jakebot12',
    'https://www.facebook.com/messages/t/nathaniel.ellingson.3',
    'https://www.facebook.com/messages/t/4dumbdogs',
    'https://www.facebook.com/messages/t/marv.thornsberry',
    'https://www.facebook.com/messages/t/benjamin.maynard.9',
    'https://www.facebook.com/messages/t/enzo.armetta',
    'https://www.facebook.com/messages/t/MCChopper',
    'https://www.facebook.com/messages/t/suzi.abramsonjohnson',
    'https://www.facebook.com/messages/t/chris.paschall.37',
    'https://www.facebook.com/messages/t/tynababbit15',
    'https://www.facebook.com/messages/t/colleen.k.cullinan',
    'https://www.facebook.com/messages/t/ivana.vilotic.7',
    'https://www.facebook.com/messages/t/mark.camacho.50',
    'https://www.facebook.com/messages/t/fellchase2.0',
    'https://www.facebook.com/messages/t/mietusmm',
    'https://www.facebook.com/messages/t/lemcvey',
    'https://www.facebook.com/messages/t/100010177366596',
    'https://www.facebook.com/messages/t/julie.davis.792',
    'https://www.facebook.com/messages/t/tblondi',
    'https://www.facebook.com/messages/t/JesusAndTheAngels',
    'https://www.facebook.com/messages/t/callie.m.cullinan',
    'https://www.facebook.com/messages/t/Valorie.PONY.Teter',
    'https://www.facebook.com/messages/t/RogerRohweder',
    'https://www.facebook.com/messages/t/caudill.619',
    'https://www.facebook.com/messages/t/1623650947',
    'https://www.facebook.com/messages/t/cindy.huelskampjudy',
    'https://www.facebook.com/messages/t/100015169534875',
    'https://www.facebook.com/messages/t/brian.daviesjones.3',
    'https://www.facebook.com/messages/t/100005972775784',
    'https://www.facebook.com/messages/t/diane.weaver.948',
    'https://www.facebook.com/messages/t/jason.hornack',
    'https://www.facebook.com/messages/t/chase.mcvey.9',
    'https://www.facebook.com/messages/t/100006728686581',
    'https://www.facebook.com/messages/t/100013570236133',
    'https://www.facebook.com/messages/t/6212130467169267713',
    'https://www.facebook.com/messages/t/tim.bangert.3',
    'https://www.facebook.com/messages/t/katie.hiro',
    'https://www.facebook.com/messages/t/armando.moreno.90',
    'https://www.facebook.com/messages/t/john.kaniecki.3',
    'https://www.facebook.com/messages/t/chris.cullinan.792',
    'https://www.facebook.com/messages/t/ryan.stoffko',
    'https://www.facebook.com/messages/t/joanna.street24',
    'https://www.facebook.com/messages/t/stephanie.desertgirl',
    'https://www.facebook.com/messages/t/rohwedernt',
    'https://www.facebook.com/messages/t/david.earl.smith',
    'https://www.facebook.com/messages/t/lemcvey',
    'https://www.facebook.com/messages/t/teresa.cvach',
    'https://www.facebook.com/messages/t/firealwaysworks',
    'https://www.facebook.com/messages/t/crystaldgraziano',
    'https://www.facebook.com/messages/t/faulkner.shane',
    'https://www.facebook.com/messages/t/colleen.k.cullinan',
    'https://www.facebook.com/messages/t/lemcvey',
    'https://www.facebook.com/messages/t/james.cullinan.315',
    'https://www.facebook.com/messages/t/jaimeneufer',
    'https://www.facebook.com/messages/t/nathaniel.ellingson.3',
    'https://www.facebook.com/messages/t/aneesh.karve',
    'https://www.facebook.com/messages/t/robert.chandler',
    'https://www.facebook.com/messages/t/robert.holweger',
    'https://www.facebook.com/messages/t/keifer.street',
    'https://www.facebook.com/messages/t/mynameisjane',
    'https://www.facebook.com/messages/t/ragunr',
    'https://www.facebook.com/messages/t/johan.larson.56',
    'https://www.facebook.com/messages/t/abe.pralle',
    'https://www.facebook.com/messages/t/crystaldgraziano',
    'https://www.facebook.com/messages/t/wolfdieter.otte',
    'https://www.facebook.com/messages/t/talbert.tso',
    'https://www.facebook.com/messages/t/abe.anwary',
    'https://www.facebook.com/messages/t/devin.west2',
    'https://www.facebook.com/messages/t/eurostar88',
    */
];

var tileWindows, jokes, windows, sendFacebookMessage, sendFacebookThanks;

function tellJokes() {
    // get multiple sets of 9 working for single page scraping in parallel
    return runSeleniumCell([
        'tile chrome windows',
        'log in facebook',
        'send facebook message',
        'send a joke',
        'list facebook threads',
        'scrape facebook profile',
        'messages from facebook',
        'send facebook thanks',
    ])
        .then(r => {
            tileWindows = r.tileWindows;
            sendFacebookMessage = r.sendFacebookMessage;
            sendJoke = r.sendJoke;
            listFacebookThreads = r.listFacebookThreads;
            sendFacebookThanks = r.sendFacebookThanks;
            return r.loginFacebook()
        })
        .then(() => listFacebookThreads())
        .then(r => {
            friends = r.slice(0, 100);
            console.log(friends);
        })
        .then(() => tileWindows(friends.slice(0, 3)))
        .then(windows => {
            var promises = [];
            for(var i = 0; i < Math.ceil(friends.length / windows.length); i++) {
                promises = promises.concat((i => windows.map((hwnd, j) => resolve => {
                    return sendFacebookThanks(friends[i * windows.length + j], hwnd).then(() => resolve());
                }))(i));
            }
            return importer.runAllPromises(promises);
        })
        .catch(e => console.log(e))
}

$$.async();
tellJokes()
    .then(r => $$.sendResult(r))
    .catch(e => $$.sendError(e))


Initializing webdriver on localhost
[ 1511490270918, 'd77ac22595709c9c2a6c2416c7f5104b' ]
[19:44:20]  COMMAND	GET 	 "/wd/hub/session/d77ac22595709c9c2a6c2416c7f5104b/window_handle"
[ 1511490270932, '5624251d09d72eefedfa6a2ea7c2c976' ]
[19:44:20]  COMMAND	GET 	 "/wd/hub/session/5624251d09d72eefedfa6a2ea7c2c976/window_handle"
[ 1511490270939, 'cc96bbbaf209cfdc18f545ac3df9f684' ]
[19:44:20]  COMMAND	GET 	 "/wd/hub/session/cc96bbbaf209cfdc18f545ac3df9f684/window_handle"
[ 1511490270946, 'ec2eca85b6fa3331aa4d685bb091fd7b' ]
[19:44:20]  COMMAND	GET 	 "/wd/hub/session/ec2eca85b6fa3331aa4d685bb091fd7b/window_handle"
[ 1511490270953, 'dbb9e06fe8f730700176ca754dc8175d' ]
[19:44:20]  COMMAND	GET 	 "/wd/hub/session/dbb9e06fe8f730700176ca754dc8175d/window_handle"
[ 1511491132315, 'd729d8d4e56c7082da83206b0b79d84f' ]
[19:44:20]  COMMAND	GET 	 "/wd/hub/session/d729d8d4e56c7082da83206b0b79d84f/window_handle"
[19:44:20]  COMMAND	POST 	 "/wd/hub/session/d729d8d4e56c7082da83206b0b79d84f/window"
[19:44:20

[19:52:09]  COMMAND	POST 	 "/wd/hub/session/d729d8d4e56c7082da83206b0b79d84f/keys"


[19:52:09]  COMMAND	POST 	 "/wd/hub/session/d729d8d4e56c7082da83206b0b79d84f/keys"


[19:52:09]  COMMAND	POST 	 "/wd/hub/session/d729d8d4e56c7082da83206b0b79d84f/keys"
[19:52:10]  COMMAND	POST 	 "/wd/hub/session/d729d8d4e56c7082da83206b0b79d84f/window"
[19:52:10]  COMMAND	GET 	 "/wd/hub/session/d729d8d4e56c7082da83206b0b79d84f/url"
[19:52:10]  COMMAND	POST 	 "/wd/hub/session/d729d8d4e56c7082da83206b0b79d84f/execute"
[19:52:10]  COMMAND	POST 	 "/wd/hub/session/d729d8d4e56c7082da83206b0b79d84f/element"
[19:52:10]  COMMAND	POST 	 "/wd/hub/session/d729d8d4e56c7082da83206b0b79d84f/element/0.634990148475949-1/click"
[19:52:11]  COMMAND	GET 	 "/wd/hub/session/d729d8d4e56c7082da83206b0b79d84f/url"
[19:52:12]  COMMAND	POST 	 "/wd/hub/session/d729d8d4e56c7082da83206b0b79d84f/execute"
[19:52:12]  COMMAND	POST 	 "/wd/hub/session/d729d8d4e56c7082da83206b0b79d84f/execute"
[19:52:12]  COMMAND	POST 	 "/wd/hub/session/d729d8d4e56c7082da83206b0b79d84f/url"
[19:52:18]  COMMAND	POST 	 "/wd/hub/session/d729d8d4e56c7082da83206b0b79d84f/execute"
[19:52:18]  COMMAND	POST 	 "/wd/hub/session/d7

[19:58:03]  COMMAND	POST 	 "/wd/hub/session/d729d8d4e56c7082da83206b0b79d84f/keys"


...

[20:04:13]  COMMAND	POST 	 "/wd/hub/session/d729d8d4e56c7082da83206b0b79d84f/keys"
[20:04:14]  COMMAND	POST 	 "/wd/hub/session/d729d8d4e56c7082da83206b0b79d84f/window"
[20:04:14]  COMMAND	GET 	 "/wd/hub/session/d729d8d4e56c7082da83206b0b79d84f/url"
[20:04:14]  COMMAND	POST 	 "/wd/hub/session/d729d8d4e56c7082da83206b0b79d84f/execute"
[20:04:14]  COMMAND	POST 	 "/wd/hub/session/d729d8d4e56c7082da83206b0b79d84f/element"
[20:04:14]  COMMAND	POST 	 "/wd/hub/session/d729d8d4e56c7082da83206b0b79d84f/element/0.9805536311721563-2/click"
[20:04:15]  COMMAND	GET 	 "/wd/hub/session/d729d8d4e56c7082da83206b0b79d84f/url"
[20:04:16]  COMMAND	POST 	 "/wd/hub/session/d729d8d4e56c7082da83206b0b79d84f/execute"
[20:04:16]  COMMAND	POST 	 "/wd/hub/session/d729d8d4e56c7082da83206b0b79d84f/execute"
[20:04:16]  COMMAND	POST 	 "/wd/hub/session/d729d8d4e56c7082da83206b0b79d84f/url"
[20:04:22]  COMMAND	POST 	 "/wd/hub/session/d729d8d4e56c7082da83206b0b79d84f/execute"
[20:04:22]  COMMAND	POST 	 "/wd/hub/session/d

[20:04:24]  COMMAND	POST 	 "/wd/hub/session/d729d8d4e56c7082da83206b0b79d84f/keys"


[20:04:24]  COMMAND	POST 	 "/wd/hub/session/d729d8d4e56c7082da83206b0b79d84f/keys"


[20:04:24]  COMMAND	POST 	 "/wd/hub/session/d729d8d4e56c7082da83206b0b79d84f/keys"
[20:04:24]  COMMAND	POST 	 "/wd/hub/session/d729d8d4e56c7082da83206b0b79d84f/window"
[20:04:25]  COMMAND	GET 	 "/wd/hub/session/d729d8d4e56c7082da83206b0b79d84f/url"
[20:04:25]  COMMAND	POST 	 "/wd/hub/session/d729d8d4e56c7082da83206b0b79d84f/execute"
[20:04:25]  COMMAND	POST 	 "/wd/hub/session/d729d8d4e56c7082da83206b0b79d84f/element"
{ Error: An element could not be located on the page using the given search parameters.
    at element("a[href*="undefined"]") - click.js:12:17
    at pause(1000) - utilities.ipynb[3]:21:10
    at click("a[href*="undefined"]") - utilities.ipynb[3]:20:10
    at execute(<Function>, ) - utilities.ipynb[3]:7:15
    at url() - getUrl.js:30:28
    at getUrl() - utilities.ipynb[3]:3:10
    at clickSpa() - orchestration.ipynb[7]:4:10
  type: 'NoSuchElement',
  message: 'An element could not be located on the page using the given search parameters.' }
Error: TypeError: Cannot read 

[ undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,

clean up old selenium sessions?

In [ ]:
var importer = require('../Core');
var execCmd = importer.import('spawn child process');
var runSeleniumCell = importer.import('selenium cell');
var createWebdriverClient = importer.import('webdriver client');

var regexToArray = (ex, str, i = 0) => {
    var co = []; var m; while ((m = ex.exec(str)) && co.push(m[i])); return co;
};

var client, sessions;
function cleanUpSessions() {
    return execCmd('docker logs act-selenium')
        .then(r => regexToArray(/\/session\/(.+?)\//ig, r.join('\n'), 1))
        .then(r => sessions = r.filter((s, i, arr) => arr.indexOf(s) === i))
        .then(() => client = createWebdriverClient())
        .then(() => {
            return importer.runAllPromises(sessions.map(session => (resolve) => {
                client.requestHandler.sessionID = session;
                client
                    .windowHandle()
                    .then(r => client.window(r.value))
                    .session()
                    .end()
                    .then(s => resolve(s.sessionId))
                    .catch(e => resolve())
            }))
        })
}
module.exports = cleanUpSessions;


In [ ]:
$$.async()
cleanUpSessions();